In [1]:
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from openpyxl import load_workbook

# capturing for database

In [3]:
def capture_images(User):
    cap = cv2.VideoCapture(0)
    count = 0
    while True:

        ret, frame = cap.read()
        count += 1
        face = cv2.resize(frame, (700, 400))

        file_name_path = f'/Users/aryan_zingade/Desktop/Att/captured/{User}_{count}.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
            
        if cv2.waitKey(1) == 5 or count == 100:
            break
            
    cap.release()
    cv2.destroyAllWindows()      
    print("Collecting Samples Complete")

In [4]:
capture_images('Sumeru')

2024-04-03 12:38:34.849 python[82102:17012441] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


# adding labels to each student

In [2]:
label_names = {'Aryan':71,'Shreyash':46, 'Kashvi':67, 'Ninad':41, 'Sakshi':30, 'Sumeru':35}
label_names

{'Aryan': 71,
 'Shreyash': 46,
 'Kashvi': 67,
 'Ninad': 41,
 'Sakshi': 30,
 'Sumeru': 35}

In [3]:
label_names.keys()

dict_keys(['Aryan', 'Shreyash', 'Kashvi', 'Ninad', 'Sakshi', 'Sumeru'])

# model training

In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def train_model(label):
	# Create lists to store the face samples and their corresponding labels
	faces = []
	labels = []
	
	# Load the images from the 'Faces' folder
	for file_name in os.listdir('/Users/aryan_zingade/Desktop/Att/captured'):
		if file_name.endswith('.jpg'):
			# Extract the label (person's name) from the file name
			name = file_name.split('_')[0]
			
			# Read the image and convert it to grayscale
			image = cv2.imread(os.path.join('/Users/aryan_zingade/Desktop/Att/captured', file_name))
			gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

			# Detect faces in the grayscale image
			detected_faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

			# Check if a face is detected
			if len(detected_faces) > 0:
				# Crop the detected face region
				face_crop = gray[detected_faces[0][1]:detected_faces[0][1] + detected_faces[0][3],
								detected_faces[0][0]:detected_faces[0][0] + detected_faces[0][2]]

				# Append the face sample and label to the lists
				faces.append(face_crop)
				labels.append(label[name])

				print(f'File Name: {file_name}, Predicted Label: {label_names[name]}')
	print(labels)

	# Train the face recognition model using the faces and labels
	recognizer = cv2.face.LBPHFaceRecognizer_create()
	recognizer.train(faces, np.array(labels))

	# Save the trained model to a file
	recognizer.save('trained_model.xml')
	return recognizer

# Train the model
Recognizer =train_model(label_names)
Recognizer


File Name: Kashvi_49.jpg, Predicted Label: 67
File Name: Kashvi_61.jpg, Predicted Label: 67
File Name: Kashvi_75.jpg, Predicted Label: 67
File Name: Sumeru_13.jpg, Predicted Label: 35
File Name: Ninad_12.jpg, Predicted Label: 41
File Name: Sumeru_6.jpg, Predicted Label: 35
File Name: Sakshi_12.jpg, Predicted Label: 30
File Name: Sakshi_3.jpg, Predicted Label: 30
File Name: Shreyash_38.jpg, Predicted Label: 46
File Name: Shreyash_10.jpg, Predicted Label: 46
File Name: Aryan_64.jpg, Predicted Label: 71
File Name: Aryan_70.jpg, Predicted Label: 71
File Name: Aryan_58.jpg, Predicted Label: 71
File Name: Aryan_59.jpg, Predicted Label: 71
File Name: Aryan_71.jpg, Predicted Label: 71
File Name: Aryan_65.jpg, Predicted Label: 71
File Name: Shreyash_11.jpg, Predicted Label: 46
File Name: Shreyash_39.jpg, Predicted Label: 46
File Name: Ninad_100.jpg, Predicted Label: 41
File Name: Sakshi_2.jpg, Predicted Label: 30
File Name: Sakshi_13.jpg, Predicted Label: 30
File Name: Sumeru_7.jpg, Predicted L

< cv2.face.LBPHFaceRecognizer 0x104154f90>

In [14]:
day = input("Enter day: ")

# sheet updation and marking

In [6]:
def update_sheet(label):
    workbook = load_workbook(filename="attendance.xlsx")
    sheet = workbook.active   

    for col in sheet.iter_cols(min_row=3, max_row=3, min_col=3, max_col=9):
        for col_cell in col:

            if col_cell.value == day:
                col_val = col_cell.column

                for row in sheet.iter_rows(min_row=4, max_row=9, min_col=2, max_col=2):
                    for row_cell in row:
                        if row_cell.value == label:

                            row_val = row_cell.row

                            # Access the cell below the 'day' column and print its value
                            cell_below = sheet.cell(row=row_val, column=col_val)
                            cell_below.value = 'P'

                            workbook.save(filename="attendance.xlsx")


# testing model by capturing new faces

In [7]:
predicted_labels = []

def test_model(model_path, test_image_path):
    # Load the trained LBPH model
    Recognizer = cv2.face.LBPHFaceRecognizer_create()
    Recognizer.read(model_path)

    test_image = cv2.imread(test_image_path)
    if test_image is None:
        print("Failed to load the test image.")
        return

    gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Iterate through the detected faces and predict the labels
    for (x, y, w, h) in faces:
        face_crop = gray[y:y+h, x:x+w]
        label, confidence = Recognizer.predict(face_crop)

        predicted_labels.append(label)

        update_sheet(label)
    
        # Display the predicted label and confidence on the image
        cv2.rectangle(test_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(test_image, f'Label: {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        cv2.putText(test_image, f'Confidence: {confidence:.2f}', (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    # Display the test image with predicted labels
    cv2.imshow('Test Image', test_image)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()


# capturing new faces

In [12]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def capture_images2():
    cap = cv2.VideoCapture(0)
    count = 0
    while True:

        ret, frame = cap.read()
        face = cv2.resize(frame, (700, 400))

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.imwrite(f'/Users/aryan_zingade/Desktop/Att/tocheck/{count}.jpg', gray[y:y + h, x:x + w])
            count += 1

        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('For ToCheck', face)
            
        if cv2.waitKey(1) == 5 or count == 50: #13 is the Enter Key
            break
            
    cap.release()
    cv2.destroyAllWindows()      
    print("Test Images Captured")

In [13]:
capture_images2()

KeyboardInterrupt: 

In [18]:
trained_model_path = 'trained_model.xml'

folder_path = '/Users/aryan_zingade/Desktop/Att/tocheck/'
files = os.listdir(folder_path)
x = len([file for file in files if file.lower().endswith('.jpg')])

for i in range(1, x+1):
    test_image_path = f'/Users/aryan_zingade/Desktop/Att/tocheck/{i}.jpg'
    test_model(trained_model_path, test_image_path)
unique_labels = set(predicted_labels)
print("Unique predicted labels:", unique_labels)

KeyboardInterrupt: 

: 

In [12]:
predicted_labels = []

def test_model(model_path, test_image_path):
    # Load the trained LBPH model
    Recognizer = cv2.face.LBPHFaceRecognizer_create()
    Recognizer.read(model_path)

    test_image = cv2.imread(test_image_path)
    if test_image is None:
        print("Failed to load the test image.")
        return

    gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Iterate through the detected faces and predict the labels
    for (x, y, w, h) in faces:
        face_crop = gray[y:y+h, x:x+w]
        label, confidence = Recognizer.predict(face_crop)

        predicted_labels.append(label)

        update_sheet(label)
    
        # Display the predicted label and confidence on the image
        cv2.rectangle(test_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        print("Predicted label = ", label)
        print("Confidence = ", confidence)
    
    cv2.imshow('Test Image', test_image)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()

trained_model_path = 'trained_model.xml'

folder_path = '/Users/aryan_zingade/Desktop/Att/tocheck/'
files = os.listdir(folder_path)
x = len([file for file in files if file.lower().endswith('.jpg')])

for i in range(1, x+1):
    test_image_path = f'/Users/aryan_zingade/Desktop/Att/tocheck/{i}.jpg'
    test_model(trained_model_path, test_image_path)
unique_labels = set(predicted_labels)
print("Unique predicted labels:", unique_labels)


71
55.73518898052329
67
55.33959849826813
71
56.03069459797418
67
59.67950938059007
71
53.462123482040504
67
58.62439966843215
30
78.64715053165543
30
79.61274968302082
35
63.00383309128237
35
70.73984670540997
30
78.6546921683993
30
77.80453677259779
30
77.18301366472407
30
79.11654715597912
35
63.67138939862844
30
69.30972948468126
35
64.86185973664702
35
65.36417897631725
30
83.27109850412509
30
74.62436183862499
30
73.34200391382514
30
70.20907835444176
35
66.62329057422936
35
61.97155693581196
35
169.3199722852315
Failed to load the test image.
Unique predicted labels: {35, 67, 30, 71}


[ WARN:0@6582.895] global loadsave.cpp:248 findDecoder imread_('/Users/aryan_zingade/Desktop/Att/tocheck/35.jpg'): can't open/read file: check file path/integrity
